In [1]:
from modules.helper import *
import modules.lyapunov as lyp
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

tolerence = 1e-1
imagefolder = 'images/'

In [5]:
import modules.main as main
import time

In [ ]:
start = time.time()
l1,l2 = main.main()
print(f'Total run time:\t{time.time()-start:.2f}')

Iterating over the transient of 1000 steps
10% done iterating
20% done iterating
30% done iterating
40% done iterating
50% done iterating
60% done iterating
70% done iterating
80% done iterating
90% done iterating
Iterating over the attractor of 100 steps
10% done iterating
20% done iterating
30% done iterating
40% done iterating
50% done iterating
60% done iterating
70% done iterating
80% done iterating
90% done iterating
Generating initial conditions
Calculating the Jacobian Matrix for each point
System set up for analysis
Calculating the Lyapunov Exponents
0% done iterating
10% done iterating
20% done iterating
30% done iterating
40% done iterating
50% done iterating
60% done iterating
70% done iterating
80% done iterating
90% done iterating
This took 0.13 seconds to run
Iterating over the transient of 1000 steps


In [26]:
l1

<xarray.DataArray (x: 10, y: 10, l: 90, a: 130)>
array([[[[-0.123855, ...,  0.      ],
         ...,
         [ 0.      , ...,  0.      ]],

        ...,

        [[ 0.      , ...,  0.      ],
         ...,
         [ 0.      , ...,  0.      ]]],


       ...,


       [[[ 0.      , ...,  0.      ],
         ...,
         [ 0.      , ...,  0.      ]],

        ...,

        [[ 0.      , ...,  0.      ],
         ...,
         [ 0.      , ...,  0.      ]]]])
Coordinates:
  * x        (x) float64 -10.0 -7.778 -5.556 -3.333 ... 3.333 5.556 7.778 10.0
  * y        (y) float64 -10.0 -7.778 -5.556 -3.333 ... 3.333 5.556 7.778 10.0
  * l        (l) float64 0.1 0.11 0.12 0.13 0.14 ... 0.95 0.96 0.97 0.98 0.99
  * a        (a) float64 0.7 0.71 0.72 0.73 0.74 ... 1.95 1.96 1.97 1.98 1.99